<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/notebooks/geemap_and_ipyleaflet.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_ipyleaflet.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_ipyleaflet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.eefolium`](https://github.com/giswqs/geemap/blob/master/geemap/eefolium.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

## Create an interactive map

In [ ]:
import geemap
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_minimap(position='bottomright')
Map

## Add tile layers

For example, you can Google Map tile layer:

In [ ]:
url = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Map', attribution='Google')

Add Google Terrain tile layer:

In [ ]:
url = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Terrain', attribution='Google')

## Add WMS layers
More WMS layers can be found at <https://viewer.nationalmap.gov/services/>.

For example, you can add NAIP imagery.

In [ ]:
url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
Map.add_wms_layer(url=url, layers='0', name='NAIP Imagery', format='image/png')

Add USGS 3DEP Elevation Dataset

In [ ]:
url = 'https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WMSServer?'
Map.add_wms_layer(url=url, layers='3DEPElevation:None', name='3DEP Elevation', format='image/png')

## Capture user inputs

In [ ]:
import geemap
from ipywidgets import Label
from ipyleaflet import Marker

Map = geemap.Map(center=(40, -100), zoom=4)

label = Label()
display(label)

coordinates = []

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'mousemove':
        label.value = str(latlon)
    elif kwargs.get('type') == 'click':
        coordinates.append(latlon)
        Map.add_layer(Marker(location=latlon))

Map.on_interaction(handle_interaction)

Map

In [ ]:
print(coordinates)

## A simpler way for capturing user inputs

In [ ]:
import geemap
Map = geemap.Map(center=(40, -100), zoom=4)
cluster = Map.listening(event='click', add_marker=True)
Map

In [ ]:
# Get the last mouse clicked coordinates
Map.last_click

In [ ]:
# Get all the mouse clicked coordinates
Map.all_clicks

## SplitMap control

In [ ]:
import geemap
from ipyleaflet import *

Map = geemap.Map(center=(47.50, -101), zoom=7)

right_layer = WMSLayer(
    url = 'https://ndgishub.nd.gov/arcgis/services/Imagery/AerialImage_ND_2017_CIR/ImageServer/WMSServer?',
    layers = 'AerialImage_ND_2017_CIR',
    name = 'AerialImage_ND_2017_CIR',
    format = 'image/png'
)

left_layer = WMSLayer(
    url = 'https://ndgishub.nd.gov/arcgis/services/Imagery/AerialImage_ND_2018_CIR/ImageServer/WMSServer?',
    layers = 'AerialImage_ND_2018_CIR',
    name = 'AerialImage_ND_2018_CIR',
    format = 'image/png'
)

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
Map.add_control(control)
Map.add_control(LayersControl(position='topright'))
Map.add_control(FullScreenControl())

Map

In [ ]:
import geemap
Map = geemap.Map()
Map.split_map(left_layer='HYBRID', right_layer='ESRI')
Map